In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
import numpy as np
import pandas as pd

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1=nn.Conv2d(1, 10, kernel_size=5)
        self.conv2=nn.Conv2d(10,20, kernel_size=5)
        self.drop=nn.Dropout2d()
        self.fc1=nn.Linear(320,50)
        self.fc2=nn.Linear(50,10)
    
    def forward(self,x):
        x=F.relu(F.max_pool2d(self.conv1(x),2))
        x=F.relu(F.max_pool2d(self.conv2(x),2))
        x=x.view(-1,320)
        x=F.relu(self.fc1(x))
        x=self.drop(x)
        x=self.fc2(x)
        return F.log_softmax(x, dim=1)

In [4]:
train_loader=torch.utils.data.DataLoader(
    datasets.MNIST("D:\Datasets\MNIST_torch",
                  train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()])
                  ),
    batch_size=1,
    shuffle=True
)

In [ ]:
train_loader.dataset[0][0]

In [ ]:
train_loader.dataset[0][0].shape

In [5]:
model=CNN()

In [6]:
model.load_state_dict(torch.load('./MNIST_CNN_model.pt'))

<All keys matched successfully>

In [7]:
test=pd.read_csv("C:\\Users\\한동훈\\Desktop\\Kaggle\\digit-recognizer\\test.csv")

x_test=np.array(test.values)
x_test=np.array([i.reshape(1,28,28) for i in x_test])

In [8]:
x_test=torch.from_numpy(x_test)

In [ ]:
x_test/255.0

In [15]:
#result=model.forward(x_test[0].unsqueeze(0)/255.0)
output=model.forward(x_test/255.0)

In [17]:
output.size()

torch.Size([28000, 10])

In [18]:
pred=output.max(1, keepdims=True)[1]

In [20]:
pred.size()

torch.Size([28000, 1])

In [21]:
pred[:10]

tensor([[2],
        [0],
        [9],
        [7],
        [3],
        [7],
        [0],
        [3],
        [0],
        [3]])

In [28]:
pd.DataFrame(pred.numpy()).to_csv('MNIST_pred_0310.csv', index=True)